# 1️⃣ 작업형1
- 빅데이터 분석기사 3회 실기
- 데이터: members.csv 또는 임의 데이터 생성

## 문제1
- 결측치 데이터(행)을 제거하고, 앞에서부터 70% 데이터만 활용해, 'f1' 컬럼 1사분위 값을 구하시오
- 데이터: members.csv

In [13]:
import pandas as pd
df = pd.read_csv("members.csv")

# print(df.isnull().sum())
# print(df.shape)

df = df.dropna()
# print(df.isnull().sum())
# print(df.shape)

# print(int(len(df)*0.7))
df = df.iloc[:int(len(df)*0.7)]
# print(df.shape)

print(df['f1'].quantile(0.25))

57.0


## 문제2
- index는 년도임. 2000년 데이터 중 2000년 평균보다 큰 값의 데이터 수
- 데이터: 아래 셀 실행

In [31]:
# 데이터 생성(먼저 실행해 주세요)
import pandas as pd
import random
random.seed(2022)
df = pd.DataFrame()
for i in range(0, 3):
    list_box = []
    for k in range(0, 200):
        ran_num = random.randint(1,200)
        list_box.append(ran_num)
    df[i+1999] = list_box
df = df.T
df

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
1999,137,74,114,140,80,150,16,133,178,181,...,124,94,118,12,50,191,137,174,56,128
2000,176,87,64,110,128,16,8,4,123,87,...,59,22,3,108,17,104,101,161,156,43
2001,128,132,123,112,134,160,180,47,115,155,...,160,111,116,75,48,51,81,101,194,73


In [32]:
# 방법 1
m = df.loc[2000].mean()
print(sum(df.loc[2000, :] > m))

# 방법 2
df = df.T
m = df[2000].mean()
print(sum(df[2000] > m))

100
100


## 문제3
- 결측치가 제일 많은 값의 컬럼명
- 데이터: members.csv

In [44]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p1/members.csv")

# 방법 1
# df = df.isnull().sum()
# df = df.sort_values(ascending=False)
# print(df.index[0])

# 방법 2
df = df.isnull().sum()
df = df.reset_index()
print(df.loc[3, 'index'])

f1


# 2️⃣ 작업형2
- 빅데이터 분석기사 3회 실기
- 제공된 데이터
    - train.csv
    - test.csv
- 예측할 컬럼 : TravelInsurance

### 시험에서 제공된 코드
~~~
import pandas as pd
a = pd.read_csv("train.csv")
b = pd.read_csv('test.csv')

pd.DataFrame(변수).to_csv('000000000.csv')
~~~


### 보험가입 확률을 묻는 문제
~~~
# csv 형태
# index,y_pred
# 0,0.152467
# 1,0.961511
# 2,0.151313
# 3,0.236632
...
# 494,0.121372
# 495,0.965155
# 496,0.986405

~~~


## 📌 My Code
- <span style='color: blue'>**TravelInsurance가 '분류' 문제이면서 '확률값'을 도출해야 함**</span>
- **roc_auc_score**: classifier가 0, 1 범주에 대해 예측한 확률값 중 범주 1에 대한 확률값 넣어줘야 함.
    - y_val_pred = model.predict_proba(X_val)[:, 1] / roc_auc_score(y_val, y_val_pred)
- **accuracy_score**: classifier가 0, 1로 예측한 값 넣어줘야 함. model.predict(X_val)

In [135]:
import pandas as pd
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/3_2/train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/3_2/test.csv")

# 1. 데이터 유형 파악
# 결측값 col: 없음 / 범주형 col: Employment Type, GraduateOrNot, FrequentFlyer, EverTravelledAbroad
# 종속변수: TravelInsurance
# print(train.info())
# print(test.info())
# print(train.shape, test.shape)

# 2. 데이터 전처리
# (1) X_full
X = train.drop(['TravelInsurance'], axis=1)
y = train['TravelInsurance']
X_full = pd.concat([X, test], axis=0)
X_full = X_full.drop(['Unnamed: 0'], axis=1)
# print(X_full.shape, y.shape)

# (2) 결측치 처리
# 결측치 없음.

# (3) 수치형 변수 스케일링 skip

# (4) 범주형 변수 인코딩
X_full = pd.get_dummies(X_full)
# print(X_full.shape)

# 3. 데이터 분리
X_train = X_full[:train.shape[0]]
X_test = X_full[train.shape[0]:]
# print(X_train.shape, X_test.shape)

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y, test_size=0.2)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

# 4. 모델 학습 및 검증
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)
y_val_pred = model.predict_proba(X_val)[:, 1]
print(y_val_pred.shape)

# 5. 모델 평가
from sklearn.metrics import roc_auc_score
roc_auc = roc_auc_score(y_val, y_val_pred)
print(roc_auc)

# 6. 결과 저장
y_pred = model.predict_proba(X_test)[:, 1]
result = pd.DataFrame({'index': X_test.index, 'y_pred':y_pred})
result.to_csv('result.csv', index=False)

# 7. 결과 확인
check = pd.read_csv('result.csv')
print(check)

(1192, 12) (298, 12) (1192,) (298,)
(298,)
0.7887109077040426
     index    y_pred
0        0  0.070000
1        1  0.009306
2        2  0.134476
3        3  1.000000
4        4  0.169833
..     ...       ...
492    492  0.020000
493    493  0.975000
494    494  0.300000
495    495  1.000000
496    496  0.990000

[497 rows x 2 columns]


In [138]:
y_test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/3_2/y_test.csv")
roc_auc_score(y_test, y_pred)

np.float64(0.7591735966735966)

# 작업형 2 풀이

## 데이터 불러오기

In [45]:
import pandas as pd
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/3_2/train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/3_2/test.csv")

## EDA

In [47]:
# 크기 확인
train.shape, test.shape

((1490, 10), (497, 9))

In [48]:
# 샘플 확인
train.head()

,Unnamed: 0,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad,TravelInsurance
0,888,28,Private Sector/Self Employed,Yes,1250000,6,1,No,No,0
1,1308,31,Private Sector/Self Employed,Yes,1250000,7,1,No,No,0
2,151,29,Private Sector/Self Employed,Yes,1200000,7,0,No,No,1
3,396,33,Government Sector,Yes,650000,6,1,No,No,1
4,135,28,Private Sector/Self Employed,Yes,800000,6,0,No,Yes,1


In [49]:
# 타입 확인
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           1490 non-null   int64 
 1   Age                  1490 non-null   int64 
 2   Employment Type      1490 non-null   object
 3   GraduateOrNot        1490 non-null   object
 4   AnnualIncome         1490 non-null   int64 
 5   FamilyMembers        1490 non-null   int64 
 6   ChronicDiseases      1490 non-null   int64 
 7   FrequentFlyer        1490 non-null   object
 8   EverTravelledAbroad  1490 non-null   object
 9   TravelInsurance      1490 non-null   int64 
dtypes: int64(6), object(4)
memory usage: 116.5+ KB


In [50]:
# 결측치 확인 train
train.isnull().sum()

Unnamed: 0             0
Age                    0
Employment Type        0
GraduateOrNot          0
AnnualIncome           0
FamilyMembers          0
ChronicDiseases        0
FrequentFlyer          0
EverTravelledAbroad    0
TravelInsurance        0
dtype: int64

In [51]:
# 결측치 확인 test
test.isnull().sum()

Unnamed: 0             0
Age                    0
Employment Type        0
GraduateOrNot          0
AnnualIncome           0
FamilyMembers          0
ChronicDiseases        0
FrequentFlyer          0
EverTravelledAbroad    0
dtype: int64

In [52]:
# 기초 통계 획인
train.describe()

,Unnamed: 0,Age,AnnualIncome,FamilyMembers,ChronicDiseases,TravelInsurance
count,1490.000000,1490.000000,1.490000e+03,1490.000000,1490.000000,1490.000000
mean,992.277181,29.600000,9.310738e+05,4.755705,0.280537,0.352349
std,566.637006,2.887829,3.763057e+05,1.603613,0.449412,0.477862
min,0.000000,25.000000,3.000000e+05,2.000000,0.000000,0.000000
25%,502.250000,28.000000,6.000000e+05,4.000000,0.000000,0.000000
50%,994.500000,29.000000,9.000000e+05,5.000000,0.000000,0.000000
75%,1479.750000,32.000000,1.250000e+06,6.000000,1.000000,1.000000
max,1983.000000,35.000000,1.800000e+06,9.000000,1.000000,1.000000


In [53]:
# 기초 통계 획인 object
train.describe(include='object')

,Employment Type,GraduateOrNot,FrequentFlyer,EverTravelledAbroad
count,1490,1490,1490,1490
unique,2,2,2,2
top,Private Sector/Self Employed,Yes,No,No
freq,1056,1270,1175,1209


In [54]:
# 타겟(레이블)
train['TravelInsurance'].value_counts()

TravelInsurance
0    965
1    525
Name: count, dtype: int64

## 데이터 전처리 및 피처엔지니어링

In [56]:
# 수치형 데이터와 범주형 데이터 분리
n_train = train.select_dtypes(exclude='object').copy()
c_train = train.select_dtypes(include='object').copy()
n_test = test.select_dtypes(exclude='object').copy()
c_test = test.select_dtypes(include='object').copy()

In [57]:
# 수치형 변수 스케일링
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
cols = ['Age', 'AnnualIncome', 'FamilyMembers', 'ChronicDiseases']
display(n_train.head())
n_train[cols] = scaler.fit_transform(n_train[cols])
n_test[cols] = scaler.transform(n_test[cols])
n_train.head()

,Unnamed: 0,Age,AnnualIncome,FamilyMembers,ChronicDiseases,TravelInsurance
0,888,28,1250000,6,1,0
1,1308,31,1250000,7,1,0
2,151,29,1200000,7,0,1
3,396,33,650000,6,1,1
4,135,28,800000,6,0,1


,Unnamed: 0,Age,AnnualIncome,FamilyMembers,ChronicDiseases,TravelInsurance
0,888,-0.25,0.538462,0.5,1.0,0
1,1308,0.50,0.538462,1.0,1.0,0
2,151,0.00,0.461538,1.0,0.0,1
3,396,1.00,-0.384615,0.5,1.0,1
4,135,-0.25,-0.153846,0.5,0.0,1


In [58]:
# 범주형 변수 인코딩
display(c_train.head())
c_train = pd.get_dummies(c_train)
c_test = pd.get_dummies(c_test)
c_train.head()

,Employment Type,GraduateOrNot,FrequentFlyer,EverTravelledAbroad
0,Private Sector/Self Employed,Yes,No,No
1,Private Sector/Self Employed,Yes,No,No
2,Private Sector/Self Employed,Yes,No,No
3,Government Sector,Yes,No,No
4,Private Sector/Self Employed,Yes,No,Yes


,Employment Type_Government Sector,Employment Type_Private Sector/Self Employed,GraduateOrNot_No,GraduateOrNot_Yes,FrequentFlyer_No,FrequentFlyer_Yes,EverTravelledAbroad_No,EverTravelledAbroad_Yes
0,False,True,False,True,True,False,True,False
1,False,True,False,True,True,False,True,False
2,False,True,False,True,True,False,True,False
3,True,False,False,True,True,False,True,False
4,False,True,False,True,True,False,False,True


In [59]:
# 분리한 데이터 다시 합침
train = pd.concat([n_train, c_train], axis=1)
test = pd.concat([n_test, c_test], axis=1)
print(train.shape, test.shape)
train.head()

(1490, 14) (497, 13)


,Unnamed: 0,Age,AnnualIncome,FamilyMembers,ChronicDiseases,TravelInsurance,Employment Type_Government Sector,Employment Type_Private Sector/Self Employed,GraduateOrNot_No,GraduateOrNot_Yes,FrequentFlyer_No,FrequentFlyer_Yes,EverTravelledAbroad_No,EverTravelledAbroad_Yes
0,888,-0.25,0.538462,0.5,1.0,0,False,True,False,True,True,False,True,False
1,1308,0.50,0.538462,1.0,1.0,0,False,True,False,True,True,False,True,False
2,151,0.00,0.461538,1.0,0.0,1,False,True,False,True,True,False,True,False
3,396,1.00,-0.384615,0.5,1.0,1,True,False,False,True,True,False,True,False
4,135,-0.25,-0.153846,0.5,0.0,1,False,True,False,True,True,False,False,True


## 검증데이터 분리

In [60]:
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train.drop('TravelInsurance', axis=1),
                                            train['TravelInsurance'],
                                            test_size=0.1,
                                            random_state=1204)
X_tr.shape, X_val.shape, y_tr.shape, y_val.shape

((1341, 13), (149, 13), (1341,), (149,))

## 모델 학습 및 예측

In [61]:
# 랜덤포레스트
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=400, max_depth=9, random_state=1204)
rf.fit(X_tr, y_tr)
pred = rf.predict_proba(X_val)[:,1]

In [62]:
# 평가
from sklearn.metrics import roc_auc_score
roc_auc_score(y_val, pred)

np.float64(0.8287755102040816)

## 예측

In [63]:
# test 예측
pred = rf.predict_proba(test)[:,1]

In [64]:
# csv 파일 생성 (예시와 다른 형태)
pd.DataFrame({'index':test.index,'y_pred':pred}).to_csv('0000.csv', index=False)

In [65]:
# csv 파일2 생성
pd.DataFrame({'y_pred':pred}).reset_index().to_csv('1111.csv', index=False)

In [68]:
# csv 파일1 확인
# pd.read_csv("0000.csv")

In [69]:
# csv 파일2 확인
# pd.read_csv("1111.csv")

## 채점
- 수험자는 알 수 없는 부분임

In [70]:
y_test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/3_2/y_test.csv")
roc_auc_score(y_test, pred)

np.float64(0.7887820512820514)

In [71]:
# 당신의 score는?: 0.7875693000693